In [1]:
#Kütüphaneler içe aktaralım
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
df=pd.read_csv("hava_durumu.csv")

In [3]:
df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14,73,9.5,82,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39,96,8.5,71,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30,64,7.0,16,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38,83,1.5,82,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27,74,17.0,66,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [4]:
df.shape

(13200, 11)

In [5]:
df["Weather Type"].value_counts()

Weather Type
Rainy     3300
Cloudy    3300
Sunny     3300
Snowy     3300
Name: count, dtype: int64

In [6]:
#Kategorik Verileri Sayısala Çevirelim
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Weather Type'] = le.fit_transform(df['Weather Type'])
df['Location'] = le.fit_transform(df['Location'])
df['Cloud Cover'] = le.fit_transform(df['Cloud Cover'])
df['Season'] = le.fit_transform(df['Season'])

In [7]:
#Bağımlı ve Bağımsız Değişkenleri Ayarlayalım
X = df.drop('Weather Type', axis=1)
y = df['Weather Type']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [9]:
dt = DecisionTreeClassifier()

In [10]:
#Karar Ağaçlarının Hiperparametreleri
print(dt.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}


In [11]:
model = dt.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9115151515151515


In [12]:
#hiperparametre uzayı için denenecek değerleri ayarlayalım
params_dt = {
    'max_depth': [None, 5, 10, 15],
    'max_features': [0.5, 0.7, 0.8, 1.0],
    'min_samples_leaf': [0.0001, 0.001, 0.01, 0.02, 0.03, 0.04]
}

#10 katlı çapraz doğruluma oluşturalım ve eğitim sonucu en iyi parametre kombinasyonunu bulalım
grid_search = GridSearchCV(estimator=dt, param_grid = params_dt,scoring = "accuracy", cv=10)
grid_search.fit(X_train, y_train)
best_hiperparam = grid_search.best_params_
print(best_hiperparam)

{'max_depth': None, 'max_features': 0.8, 'min_samples_leaf': 0.001}


In [13]:
#test verileri için doğruluk
best_cv_score = grid_search.best_score_
best_model = grid_search.best_estimator_
test_dogruluk = best_model.score(X_test, y_test)
test_dogruluk

0.9063636363636364

In [14]:
#eğitim verileri için doğruluk
best_cv_score = grid_search.best_score_
best_model = grid_search.best_estimator_
train_dogruluk = best_model.score(X_train, y_train)
train_dogruluk

0.9413131313131313